## Track Evaluation Results in Azure AI Studio

Contoso Gameworks is developing an AI-powered dialogue generator for video game characters, customizing interactions based on game scenarios. The generator should be evaluated for relevance (fitting dialogue to the game's plot and character traits), fluency (smooth, immersive conversations), and risk and safety (ensuring no violent, offensive, or unfair language is introduced).

In this exercise, you will run an evaluation to assess a dataset of character dialogue generated by the generator. You will push the results to your Azure AI project to track the results in Azure AI Foundry.

## Sign-in to Azure

To track the evaluation results in Azure AI Studio, you'll need to first login with your Azure AI account used to provision the Azure resources.

Open a new terminal and enter the following command and follow the instruction in the terminal:

`az login --use-device-code`

Once you've logged in, select your subscription in the terminal.

## Retrieve your values for assigning the Storage Blob Data Contributor role

You'll need the following information to later assign yourself the **Storage Blog Data Contributor** role, which provides access to the Azure AI Project storage account. This permission is necessary for pushing the results to your Azure AI project.

**Resource Group and Subscription ID**

Each value can be found within the **Management center** for your project, located in [Azure AI Foundry](https://ai.azure.com). The **Management center** page is accessible via the left navigation menu of your project.

**User ID**

Enter the following command into the terminal:

`az ad signed-in-user show --query id --output tsv`

## Assign yourself the Storage Blob Data Contributor role

In the terminal, enter the following command, replacing the placeholder text with your **subscription ID**, **resource group**, and **user ID**.

`az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/<mySubscriptionID>/resourceGroups/<myResourceGroupName> --assignee-principal-type User --assignee-object-id "<user-id>"`


## Install the package

The evaluator classes as well as the `evaluate` function are available in the Azure AI Evaluation SDK. In addition, we'll need to use `promptflow[azure]` to track the results in our Azure AI project. We'll begin by installing all the required packages.

In [ ]:
%pip install promptflow-azure azure-ai-evaluation

## Import packages

We'll import `os` to access our environment variables and `json` to later dump the results into a new file. We'll also need `Path` to access the dataset. And finally, we'll need to import `evaluate` and the evaluators that we'll later use for evaluation.

In [ ]:
import os
import json
from pathlib import Path
from azure.ai.evaluation import evaluate, RelevanceEvaluator, FluencyEvaluator, ViolenceEvaluator, HateUnfairnessEvaluator, SexualEvaluator, SelfHarmEvaluator
from azure.identity import DefaultAzureCredential

## Set environment variables to create an instance of the evaluators

We'll now set the environment variables that'll be required to create an instance of the evaluators. You'll need the following:

- Azure OpenAI endpoint
- Azure OpenAI API Key
- Azure deployment

You can locate your **Azure OpenAI endpoint** and **Azure OpenAI API Key** by navigating to **Models + endpoints**, selecting the model, and copying the respective credentials for your model deployment.

In [ ]:
os.environ['AZURE_OPENAI_ENDPOINT'] = 'Your OpenAI endpoint'
os.environ['AZURE_OPENAI_API_KEY'] = 'Your OpenAI API key'
os.environ['AZURE_OPENAI_DEPLOYMENT'] = 'Your Azure OpenAI API key'

## Configure the model_config

The `model_config` is necessary as it's a required parameter when creating an instance of an evaluator class. Let's configure the `model_config` with the following:

- Azure OpenAI endpoint
- Azure OpenAI API key

In [ ]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

## Configure the Azure AI project

We'll now set the evironment variables that'll be required to configure the `azure_ai_project`. The `azure_ai_project` is later passed into the `ContentSafetyEvaluator` to create an instance of the evaluator class. 

In addition to some of the environment variables that have been configured, you'll also need the following:

- Azure project name
- Resource group name
- Subscription ID

Each value can be found within the **Management center** for your project, located in [Azure AI Foundry](https://ai.azure.com). The **Management center** page is accessible via the left navigation menu of your project.

Let's first create environment variables for the variables listed above.


In [ ]:
os.environ['AZURE_PROJECT_NAME'] = 'Your project name'
os.environ['RESOURCE_GROUP_NAME'] = 'Your resource group name'
os.environ['SUBSCRIPTION_ID'] = 'Your subscription ID'

And now, let's configure the `azure_ai_project`.

In [ ]:
azure_ai_project = {
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
    "resource_group_name": os.environ.get("RESOURCE_GROUP_NAME"),
    "subscription_id": os.environ.get("SUBSCRIPTION_ID"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "api_version": os.environ.get("OPENAI_API_VERSION")
}

## Set the path of the dataset

We'll now set the path of the dataset that we'll use for the evalutation. The dataset is within the `gameworks-dialog.jsonl` file and consists of the following values for each row of data:

- query
- response
- context

In [ ]:
path = "gameworks-dialog.jsonl"

## Create an instance of the evaluators

Let's now create an instance of the **Relevance**, **Fluency** and **Content Safety** evaluators. The **Content Safety** evaluator is a composite evaluator which combines the following evaluators:

- `ViolenceEvaluator`
- `SexualEvaluator`
- `SelfHarmEvaluator`
- `HateUnfairnessEvaluator`

In [ ]:
relevance_eval = RelevanceEvaluator(model_config)
fluency_eval = FluencyEvaluator(model_config)
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

## Create the call to evaluate the dataset

We can run an evaluation for a dataset with the `evaluate` function and include our list of evaluators. We must also ensure that the `evaluator_config` is set with the appropriate parameters and values for the `query`, `response`, `context` and `ground_truth`.

Since we want to track the evaluation results in our Azure AI project, we'll need to include the Azure AI Foundry project information.

In [ ]:
result = evaluate(
    data=path,
    evaluators={
        "relevance": relevance_eval,
        "fluency": fluency_eval,
        "violence": violence_eval,
        "hate_unfairness": hateunfairness_eval,
        "sexual": sexual_eval,
        "self_harm": selfharm_eval
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    },
    azure_ai_project = azure_ai_project
)

## View the results in Azure AI Foundry

Now that the evaluation is complete, you can navigate to the **Evaluation** section of the Azure AI Foundry portal to view the results. Alternatively, you can output a link to the evaluation location using the `studio_url` returned from running `evaluate`.

In [ ]:
print(result['studio_url'])

## Delete resources

If you've finished exploring Azure AI Services, delete the Azure resource that you created during the workshop.

**Note**: You may be prompted to delete your deployed model(s) before deleting the resource group.